In [13]:
import sys
import os

notebook_dir = os.path.dirname(os.path.abspath("__file__"))  # Répertoire du notebook
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))  # Répertoire parent contenant "src"
sys.path.append(project_root)

# Importer les modules
from src import Graph, load_graph_from_osm, generate_random_graph

In [16]:
g = Graph()

g.draw_country_contours("France")

AttributeError: 'Graph' object has no attribute 'draw_country_contours'